In [1]:
import pandas as pd
import os
import requests as req
import json
import numpy as np
import datetime
from matplotlib import style
import matplotlib
%matplotlib inline
style.use('ggplot')

# This is to import custom-made modules

# This can be removed after making these modules a real library
import os, sys
#lib_path = os.path.abspath(os.path.join('..', 'src')) # relative path of the source code in Box Folder 
#sys.path.append(lib_path)

In [2]:
data = pd.read_csv(os.path.join(".","Units-1311.csv")) 

In [3]:
data['opponent'].unique()

array(['Z', 'T', 'P'], dtype=object)

In [4]:
#data2 = pd.read_csv(os.path.join("..","lossreplay.csv")) 

data.replace({'opponent' : { 'T' : 1, 'Z' : 2, 'P' : 3 }}, inplace =True)


In [5]:
data.head()

,winner,opponent,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,Ghost,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
datatoTest = data.iloc[12:95, 1:] #split this up to test (first game)
datatoModel = data.iloc[96:, :] #

In [7]:
#datatoTest.to_csv("parsed_replay.csv")

In [8]:
#datatoModel.to_csv("1Units-1311.csv")

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X = datatoModel.iloc[:,1:]
y = datatoModel['winner']

In [7]:
#X = data.iloc[:,1:]
#y  = data['winner']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [63]:
from sklearn.neighbors import KNeighborsClassifier

In [64]:
knn = KNeighborsClassifier(n_neighbors = 10)

In [65]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [66]:
knn.score(X_test, y_test)

0.8086727751065953

In [67]:
X_test.head()

,opponent,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,Ghost,SupplyDepot,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
111502,2,224,0,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
50244,2,1229,2,0,0,0,15,28,0,12,...,25,6,2,0,1,1,0,0,5,0
1827,3,709,3,0,0,0,7,2,0,1,...,19,0,2,0,0,2,0,6,2,0
411513,2,724,0,12,4,0,1,0,0,6,...,10,0,3,0,1,1,0,0,0,0
466583,1,929,3,0,7,0,6,16,0,10,...,19,14,0,3,0,0,2,2,2,0


In [68]:
y_test.head()

111502    0
50244     1
1827      0
411513    1
466583    1
Name: winner, dtype: int64

In [69]:
X_test.shape

(121253, 54)

In [70]:
#forMatt = data.iloc[2:50,1:]

In [71]:
predictions = knn.predict_proba(datatoTest)

In [72]:
predictions[0][1]

0.4

In [73]:
predictions[2][1]

0.6

In [74]:
len(predictions)

83

In [75]:
mylist =[]
for i in xrange(len(predictions)):
    mylist.append(predictions[i][1])

In [76]:
datatoTest['prediction'] = mylist

In [85]:
#this is the average probability of the player winning the game (which in this case is true)
reduce(lambda x, y: x + y, mylist) / len(mylist)

0.38674698795180734

In [77]:
toplot = pd.DataFrame()
toplot['seconds'] = datatoTest['seconds']
toplot['prediction'] = datatoTest['prediction']
toplot.head()

,seconds,prediction
12,0,0.4
13,20,0.7
14,37,0.6
15,54,0.4
16,63,0.2


In [80]:
toplot.to_csv('forMatt.csv', index=False, header=False)

In [30]:
import pandas as pd
import requests as req
import numpy as np
import datetime
import os, sys

#importing the modules to split the data into train and test sets 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

neighbors_values = [5,10,15,20]

# Uploading all the win/lose replays into dataframes to make it more manegable
data = pd.read_csv(os.path.join(".", "1Units-1311.csv"))

#this needs to be in the same folder as thr KNN.py file
game_data = pd.read_csv(os.path.join(".", "parsed_replay.csv"), index_col=0)

#game_data = game_data.iloc[:,1:] #if we use the parsing from replay we need to
#get rid of the column that says if the game was a win or loss
#replace the value of who they played against

#game_data.replace({'opponent' : { 'T' : 1, 'Z' : 2, 'P' : 3 }}, inplace =True)

#replacing the column of oppponent from strings to ints, pretty self explanitory
#data.replace({'opponent' : { 'T' : 1, 'Z' : 2, 'P' : 3 }}, inplace =True)

#we are splitting up the data into the X values and y values 
# X is the inputs which is composed of various pieces of information
#y is the output which in our case is weather the player won or lost

#X = data.iloc[:,1:]
#y = data['winner']

# the method train_test_split will split the dataframe into random train and test subsets 
#X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

# KNeighborsClassifier params: n_neighbors, weights, algorithm, leaf_size, p, metric, metric_params, n_jobs
# see http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html for more info on params
#




In [32]:
game_data.head()

,opponent,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,Ghost,SupplyDepot,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
12,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,2,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,2,37,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,2,63,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [33]:
data.head()

,Unnamed: 0,winner,opponent,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
0,96,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,97,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,98,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,99,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
for num_neighbors in neighbors_values:
    print("starting")
    parsedData = data.copy()
    #parsedData.copy(data)
    #display(data.head())
    X = parsedData.iloc[:,1:]
    y = parsedData['winner']
    print("post loc")
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)
    testingdata = game_data.copy()
    #testingdata = game_data
    knn = KNeighborsClassifier(n_neighbors=num_neighbors)
    knn.fit(X_train, y_train)
    predictions = knn.predict_proba(testingdata)
    prediction_list = []
    
    print(num_neighbors)
    #predictions is formatted in a 2D array where the second value of the row is
    #the prediction of winning ie( predictions=[[0.7,0.3],[0.1,0.9]] ) where
    # the probability of winning is 0.3 and 0.9
    for i in xrange(len(predictions)):
        prediction_list.append(predictions[i][1])

    #appending a column of the predictions we are making throughout the game
    testingdata['prediction'] = prediction_list
    
    #this gives us the entire average probability of the entire game winning
    #if we want to check out how well our prediction did for the whole game
    probability_of_entire_game_win = reduce(lambda x, y: x + y, prediction_list)/len(prediction_list)

    #creating a smaller dataframe that will only contain the seconds of the game
    # and the prediction for each of those time steps
    data_for_plotting = pd.DataFrame()
    data_for_plotting['seconds'] = testingdata['seconds']
    data_for_plotting['prediction'] = testingdata['prediction']

    # send the prediction to a CSV file with the unique name of K number of 
    #neighbors as the name of the file and removing indexes and the column names
    filename = "K_"+str(num_neighbors) + ".csv"
    data_for_plotting.to_csv(filename, index = False, header = True)

starting
post loc
5
starting
post loc
10
starting
post loc
15
starting
post loc
20
